# Visualization of Stock Trading

In [1]:
# import libraries
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns

## Set relevant parameters

In [34]:
train_stock = 'data/GOOGL.csv'
val_stock = 'data/GOOGL_2018.csv'
window_size = 10
batch_size = 16
ep_count = 10
model_name = 'model_GOOGL'
pretrained = False
debug = False

## Load Dataset into pandas DataFrame

In [35]:
# read csv into dataframe
df = pd.read_csv(val_stock)

# filter out the desired features
df = df[['Date', 'Adj Close']]

# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})

df.head()

,date,actual
0,2017-08-10,923.590027
1,2017-08-11,930.090027
2,2017-08-14,938.929993
3,2017-08-15,938.080017
4,2017-08-16,944.270020


In [36]:
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

In [37]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 2 columns):
date      253 non-null datetime64[ns]
actual    253 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 4.0 KB


,date,actual
0,2017-08-10,923.590027
1,2017-08-11,930.090027
2,2017-08-14,938.929993
3,2017-08-15,938.080017
4,2017-08-16,944.270020


## Visualization of actions taken by Trading Bot

In [38]:
def visualize(df, history):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['predicted'])) - 50, max(max(df['actual']), max(df['predicted'])) + 50), clamp=True)
    
    # plot a line chart for actual stock prices
    actual = alt.Chart(df).mark_line(
        color='red',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('actual', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    ).properties(height=300, width=1000)
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point().encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=val_stock).properties(height=300, width=1000)
    
    return chart

## Training the model

In [39]:
import logging
import coloredlogs

from train import *
from evaluate import show_eval_result

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

agent = Agent(window_size, pretrained=pretrained, model_name=model_name)
train_data = get_stock_data(train_stock)
val_data = get_stock_data(val_stock)

initial_offset = val_data[1] - val_data[0]

for episode in range(1, ep_count + 1):
    train_result = train_model(agent, 1, train_data, ep_count=ep_count,
                                batch_size=batch_size, window_size=window_size)
    val_result, _ = evaluate_model(agent, val_data, window_size, debug)
    show_train_result(train_result, val_result, initial_offset)

2018-12-31 17:27:16 ubuntu root[32111] DEBUG switching to TensorFlow for CPU
Episode 1/10: 100%|██████████| 1760/1760 [00:20<00:00, 85.98it/s]
2018-12-31 17:27:37 ubuntu root[32111] INFO Episode 1/10 - Train Position: +$473.45  Val Position: +$1.15  Train Loss: 0.5512  (~37.2028 secs)
Episode 1/10: 100%|██████████| 1760/1760 [00:19<00:00, 88.58it/s]
2018-12-31 17:27:56 ubuntu root[32111] INFO Episode 1/10 - Train Position: +$1204.55  Val Position: -$135.77  Train Loss: 0.2382  (~34.9744 secs)
Episode 1/10: 100%|██████████| 1760/1760 [00:20<00:00, 84.35it/s]
2018-12-31 17:28:17 ubuntu root[32111] INFO Episode 1/10 - Train Position: +$705.36  Val Position: +$25.10  Train Loss: 0.4046  (~34.3783 secs)
Episode 1/10: 100%|██████████| 1760/1760 [00:19<00:00, 90.11it/s]
2018-12-31 17:28:37 ubuntu root[32111] INFO Episode 1/10 - Train Position: +$596.32  Val Position: +$1584.92  Train Loss: 0.4007  (~36.4967 secs)
Episode 1/10: 100%|██████████| 1760/1760 [00:19<00:00, 90.57it/s]
2018-12-31 17:

## Plotting the evaluation actions

In [ ]:
val_result, history = evaluate_model(agent, val_data, window_size, debug)
show_eval_result(model_name, val_result, initial_offset)
chart = visualize(df, history)
chart